In [2]:

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from Herramientas import teselator as ts
import numpy as np
%matplotlib inline

# BASES DE DATOS

In [7]:
bd_mz_rm = ('manzanas_rm_AMS.shp')
gdf_mz_rm = gpd.read_file(bd_mz_rm)

DriverError: Unable to open manzanas_rm_AMS.shx or manzanas_rm_AMS.SHX. Set SHAPE_RESTORE_SHX config option to YES to restore or create it.

In [2]:
display(gdf_mz_rm)

NameError: name 'gdf_mz_rm' is not defined

In [ ]:
gdf_mz_rm.columns

In [ ]:
#gdf_mz_rm = gdf_mz_rm.loc[gdf_mz_rm['REGION'] == '13']

In [ ]:
Total1 = gdf_mz_rm['TOTAL_PERS'].sum()
print ("Total Personas:",Total1)

In [ ]:
bd_mz_rm1 = ('manzanas_rm.shp')
gdf_mz_rm1 = gpd.read_file(bd_mz_rm1)

# CELDAS

In [ ]:
xmin, ymin, xmax, ymax = gdf_mz_rm1.total_bounds

In [3]:
celdas_rm = ts.hexagon_teselation(xmin, xmax, ymin, ymax, 549.8183, 32719) #Los 549.8183 son el radio de la celda
celdas_rm['centers'] = celdas_rm.centroid
ax = celdas_rm.plot(edgecolor = 'black', figsize = (15,15))
gdf_mz_rm.plot(ax = ax, color = 'g', edgecolor = 'r', alpha = 0.5)
celdas_rm.head()

NameError: name 'ts' is not defined

In [ ]:
display(celdas_rm)

In [ ]:
def suma_superficies(rol, df):
    a_revisar = df.loc[df['FID']==rol]
    suma= a_revisar['TOTAL_PERS'].sum()
    return suma

gdf_mz_rm['Población'] = gdf_mz_rm.apply(lambda row:suma_superficies(row['FID'],gdf_mz_rm) ,axis=1)

gdf_mz_rm.head()

In [ ]:
display(gdf_mz_rm)

In [ ]:
P=gdf_mz_rm.plot(column='Población',figsize = (15,15), legend=True)
P.set_title('Población en Región Metropolitana de Santiago, Chile', fontsize=12)

In [ ]:
#gdf_mz_rm.to_file('poblacion_rm_549.shp')

In [ ]:
#Intersección celda con manzana
inter = gpd.overlay(celdas_rm, gdf_mz_rm, how='intersection')
inter.head()

In [ ]:
def proporcion(area_celda,FID,gdf_mz_rm):
    mz = gdf_mz_rm.loc[gdf_mz_rm['FID']==FID]
    area_mz = mz.area
    return area_celda/area_mz.sum()
inter['proporcion'] = inter.apply(lambda row: proporcion(row['geometry'].area,row['FID'],gdf_mz_rm), axis =1)
inter.head()

In [ ]:
inter['m2 prop'] = inter.apply(lambda row: row['proporcion']*row['TOTAL_PERS'], axis =1)
inter.head()

In [ ]:
def union_superficies(id_,inter):
    filas = inter.loc[inter['cell']==id_]
    m2 = filas[f'm2 prop'].sum()
    return m2

celdas_rm['M2_Poblacion_Celdas_549'] = celdas_rm.apply(lambda row: union_superficies(row['cell'],inter), axis =1)

# Muchas son 0 porque no están en la intersección
display(celdas_rm)

In [ ]:
Total1 = celdas_rm['M2_Poblacion_Celdas_549'].sum()
print ("Total Personas:",Total1)

In [ ]:
celdas_rm = celdas_rm.drop(["centers"], axis = 1)

In [ ]:
celdas_rm.to_file("poblacion_549.shp")